# Analyze $d_0*\mathrm{charge}$ distributions.

In [ ]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('/Users/Jake/')
sys.path.append('/Users/Jake/HiggsMassMeasurement/')
sys.path.append('/Users/Jake/HiggsMassMeasurement/d0_Studies/')

# Neat tricks.
from itertools import chain
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Rectangle
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Local imports. 
from PyUtils.Utils_Files import makeDirs, make_str_title_friendly, check_overwrite
from PyUtils.Utils_Plotting import (change_cmap_bkg_to_white, save_plots_to_outpath, make_1D_dist, get_stats_1Dhist, 
                                    get_stats_2Dhist, hist_y_label, make_2by2_subplots_for_ratioplots,
                                    add_underoverflow_entries, make_stats_legend_for_1dhist, make_stats_legend_for_2dhist, 
                                    make_stats_legend_for_gaus_fit)
from PyUtils.Utils_Physics import theta2pseudorap, pseudorap2theta, calc_dR, calc_dphi
from PyUtils.Utils_StatsAndFits import gaussian, fit_with_gaussian, iterative_fit_gaus
from PyUtils.Utils_Collection_Helpers import weave_lists
# from d0_Utils.d0_cls import KinemBinnedEtaPt
from d0_Utils.d0_fns import (make_binning_array, shift_binning_array, get_subset_mask, make_kinem_subplot)
from d0_Utils.d0_dicts import color_dict, label_LaTeX_dict

plt.rcParams.update({'figure.max_open_warning': 10})    # Related to saving memory and opening plots.

# pd.options.display.max_columns = 23
pd.options.display.max_columns = None

plt.style.use('cmsstyle_plot')
# plt.style.use("grid_multiple_plots")

In [ ]:
%%time
infile_path_MC_2016 = '/Users/Jake/Desktop/MC_2016.h5'
infile_path_MC_2017 = '/Users/Jake/Desktop/Research/Higgs_Mass_Measurement/NTuples/MC/MC_2017.feather'
infile_path_MC_2018 = '/Users/Jake/Desktop/Research/Higgs_Mass_Measurement/NTuples/MC/MC_2018.feather'

# df_MC_2016 = pd.read_feather(infile_path_MC_2016)
# df_MC_2017 = pd.read_feather(infile_path_MC_2017)
# df_MC_2018 = pd.read_feather(infile_path_MC_2018)

df_MC_2016 = pd.read_hdf(infile_path_MC_2016)
# df_MC_2017 = pd.read_hdf(infile_path_MC_2017)
# df_MC_2018 = pd.read_hdf(infile_path_MC_2018)

# Specific kbins for main analysis.

In [ ]:
class KinemBinnedEtaPt():

    def __init__(self, df_original, n_evts, 
                 massZ_cut_ls, eta_cut_ls, pT_cut_ls, d0q_cut_ls, d0_type="BS", dR_cut=0.02, 
                 use_ptotal_instead=False, verbose=False):
        """
        Pass in a DataFrame (DF) and specify the eta and pT cuts to create a subset of DF.
        
        Parameters
        ----------
        df_original : pandas.DataFrame
            ROOT file converted into a DataFrame. Columns are branches. Rows are events.
        n_evts : int
            Number of events to search over - not guaranteed to find this many events! 
            Use '-1' to loop over all events in the df. 
        eta_cut_ls : list or array-like of floats
            A list of [eta_min, eta_max]. Example: [0.9, 1.8]
        pT_cut_ls : list or array-like of floats
            A list of [pT_min, pT_max]. Example: [5, 20]
        d0q_cut_ls : list or array-like of floats
            A list of values of d0*charge to cut on: [d0q_min, d0q_max]. E.g. [-0.01, 0.01]
        d0_type : str
            Which d0 to cut on: "BS" or "PV"
        use_ptotal_instead : bool
            Cut on total momentum instead of pT. 
            In most places, pT could stand for either p or pT (depending on 'use_ptotal_instead').
                Just be careful because not all places are adapted for p!
        dR_cut : float
            A cut to save events in which muon1 and muon2 both have dR < dR_cut.
        verbose : bool
            If True, get debug info and see where you are while the code runs.
            
        NOTE:
            The methods further down are more developed than the methods closer to __init__(). 
            Therefore, clean up and consolidate the earlier methods. 
        """
        if n_evts == -1:
            n_evts = len(df_original)
        df = df_original.loc[:n_evts].copy()    # Original DF. 
        
        self.n_evts_asked_for = n_evts
        self.n_evts_found = -999
        self.kinem_vals_after_selection = {}
        self.stats_dict = {}
        
        self.cuts = ""
        self.use_ptotal_instead = use_ptotal_instead
        self.p_str = "p" if (self.use_ptotal_instead) else "pT"
        self.p_str_latex = r"$p^{\mathrm{REC}}$" if (self.use_ptotal_instead) else r"$p_{T}^{\mathrm{REC}}$"

        self.massZ_min = massZ_cut_ls[0]
        self.massZ_max = massZ_cut_ls[1]        
        self.eta_min   = eta_cut_ls[0]
        self.eta_max   = eta_cut_ls[1]
        self.pT_min    = pT_cut_ls[0] 
        self.pT_max    = pT_cut_ls[1]         
        self.d0q_min   = d0q_cut_ls[0]
        self.d0q_max   = d0q_cut_ls[1]
        self.d0_type   = d0_type
        self.dR_cut    = dR_cut    
        
        self.apply_initial_cuts(df, verbose)
        self.save_exclusive_masks()
        
    def apply_initial_cuts(self, df, verbose=False):
        """
        Creates a subset of the original DataFrame in which initial cuts are applied.
        Cuts:
            pT
            eta
            d0
            massZ
            dR
        """
        # Cuts:
        eta_min   = self.eta_min  
        eta_max   = self.eta_max  
        pT_min    = self.pT_min   
        pT_max    = self.pT_max   
        d0q_min    = self.d0q_min
        d0q_max    = self.d0q_max
        dR_cut    = self.dR_cut   
        massZ_min = self.massZ_min
        massZ_max = self.massZ_max

        #----------------#
        #--- Analysis ---#
        #----------------#
        # GEN info. 
        eta1_gen_ser = df['genLep_eta1']
        eta2_gen_ser = df['genLep_eta2']
        phi1_gen_ser = df['genLep_phi1']
        phi2_gen_ser = df['genLep_phi2']
        pT1_gen_ser  = df['genLep_pt1']
        pT2_gen_ser  = df['genLep_pt2']

        # RECO info.
        eta1_rec_ser = df['eta1']
        eta2_rec_ser = df['eta2']
        phi1_rec_ser = df['phi1']
        phi2_rec_ser = df['phi2']
        pT1_rec_ser  = df['pT1']
        pT2_rec_ser  = df['pT2']
        
        # Store other variables.
        df.loc[:,'genLep_theta1'] = theta1_gen_ser = pseudorap2theta(eta1_gen_ser)
        df.loc[:,'genLep_theta2'] = theta2_gen_ser = pseudorap2theta(eta2_gen_ser)
        df.loc[:,'theta1'] = theta1_rec_ser = pseudorap2theta(eta1_rec_ser)
        df.loc[:,'theta2'] = theta2_rec_ser = pseudorap2theta(eta2_rec_ser)
        
        df.loc[:,'genLep_p1'] = pT1_gen_ser / np.sin( theta1_gen_ser )
        df.loc[:,'genLep_p2'] = pT2_gen_ser / np.sin( theta2_gen_ser )
        df.loc[:,'p1'] = pT1_rec_ser / np.sin( theta1_rec_ser )  # Total momentum
        df.loc[:,'p2'] = pT2_rec_ser / np.sin( theta2_rec_ser )  # Total momentum
                        
        df.loc[:,'delta_eta1'] = deta1_ser = eta1_rec_ser - eta1_gen_ser
        df.loc[:,'delta_eta2'] = deta2_ser = eta2_rec_ser - eta2_gen_ser
        df.loc[:,'delta_theta1'] = dtheta1_ser = theta1_rec_ser - theta1_gen_ser
        df.loc[:,'delta_theta2'] = dtheta2_ser = theta2_rec_ser - theta2_gen_ser

        # Remember that delta_phi requires special treatment:
        # -pi < delta_phi < pi
        df.loc[:,'delta_phi1'] = dphi1_ser = calc_dphi(phi1_rec_ser, phi1_gen_ser)
        df.loc[:,'delta_phi2'] = dphi2_ser = calc_dphi(phi2_rec_ser, phi2_gen_ser)

        df.loc[:,'delta_R1'] = dR1_ser = calc_dR(deta1_ser, dphi1_ser)
        df.loc[:,'delta_R2'] = dR2_ser = calc_dR(deta2_ser, dphi2_ser)
        df.loc[:,'delta_Rtheta1'] = dRtheta1_ser = calc_dR(dtheta1_ser, dphi1_ser)
        df.loc[:,'delta_Rtheta2'] = dRtheta2_ser = calc_dR(dtheta2_ser, dphi2_ser)
        
        df.loc[:,'delta_pT1'] = dpT1 = pT1_rec_ser - pT1_gen_ser
        df.loc[:,'delta_pT2'] = dpT2 = pT2_rec_ser - pT2_gen_ser
        
        df.loc[:,'delta_pToverpT1'] = dpT1 / pT1_gen_ser
        df.loc[:,'delta_pToverpT2'] = dpT2 / pT2_gen_ser
        
        df.loc[:,'delta_pToverRecpT1'] = dpT1 / pT1_rec_ser
        df.loc[:,'delta_pToverRecpT2'] = dpT2 / pT2_rec_ser     
        
        df.loc[:,'d0BSq1'] = df['d0BS1'] * df['Id1'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0BSq2'] = df['d0BS2'] * df['Id2'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0PVq1'] = df['d0PV1'] * df['Id1'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0PVq2'] = df['d0PV2'] * df['Id2'].replace(13,-1).replace(-13,1)
        
        # Create masks.
        self.mask_massZ = mask_massZ = self.get_mask_massZ(df)
        
        self.mask_eta1, self.mask_eta2 = mask_eta1, mask_eta2 = self.get_mask_eta(df)
        self.mask_pT1,  self.mask_pT2  = mask_pT1,  mask_pT2  = self.get_mask_pT(df)
        self.mask_d0q1, self.mask_d0q2 = mask_d0q1, mask_d0q2 = self.get_mask_d0q(df)    
        self.mask_dR1,  self.mask_dR2  = mask_dR1,  mask_dR2  = self.get_mask_dR(df) 
        
        # Combine masks.
        mask_kinembin_lep1 = mask_dR1 & mask_eta1 & mask_pT1 & mask_d0q1
        mask_kinembin_lep2 = mask_dR2 & mask_eta2 & mask_pT2 & mask_d0q2

        # Keep all events in which either muon1 passed all selections or muon2 passed all. 
        all_masks = mask_massZ & (mask_kinembin_lep1 | mask_kinembin_lep2)
        
        # Apply masks and update DataFrame.
        self.binned_df = df[all_masks]
        
        self.n_evts_found = len(self.binned_df)
        
        if (verbose): 
            perc = self.n_evts_found / float(self.n_evts_asked_for) * 100.
            print(r"Events found: {} ({:.2f}% of total events), using cuts: {}".format(self.n_evts_found, perc, self.cuts))
            
    def save_exclusive_masks(self):
        """
        Save two boolean masks:
            mask 1 -> shows which events muon1 passes
            mask 2 -> shows which events muon2 passes
        Keeping the masks separate like this is what is "exlusive" about the masks. 
        """
        df = self.binned_df
    
        # Create Masks
        if (self.use_ptotal_instead):
            mask_pT1 = (self.pT_min < df['p1']) & (df['p1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['p2']) & (df['p2'] < self.pT_max)
        else:
            mask_pT1 = (self.pT_min < df['pT1']) & (df['pT1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['pT2']) & (df['pT2'] < self.pT_max)
        
        mask_eta1 = (self.eta_min < abs(df['eta1'])) & (abs(df['eta1']) < self.eta_max)
        mask_eta2 = (self.eta_min < abs(df['eta2'])) & (abs(df['eta2']) < self.eta_max)        

        # Combine masks and save them.
        self.mask_kinembin_lep1 = mask_eta1 & mask_pT1
        self.mask_kinembin_lep2 = mask_eta2 & mask_pT2
      
    def get_mask_d0q(self, df):
        if self.d0_type == "PV":
            mask_d0q1 = (self.d0q_min < df['d0PVq1']) & (df['d0PVq1'] < self.d0q_max)
            mask_d0q2 = (self.d0q_min < df['d0PVq2']) & (df['d0PVq2'] < self.d0q_max)
        elif self.d0_type == "BS":
            mask_d0q1 = (self.d0q_min < df['d0BSq1']) & (df['d0BSq1'] < self.d0q_max)
            mask_d0q2 = (self.d0q_min < df['d0BSq2']) & (df['d0BSq2'] < self.d0q_max)
        self.cuts += "\n" + r",  $%.3f < d_{0}^{\mathrm{%s}}*q(\mu) < %.3f$" % (self.d0_min, self.d0_type, self.d0_max)
        return mask_d0q1, mask_d0q2
        
    def get_mask_pT(self, df):
        if (self.use_ptotal_instead):
            mask_pT1 = (self.pT_min < df['p1']) & (df['p1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['p2']) & (df['p2'] < self.pT_max)
        else:
            mask_pT1 = (self.pT_min < df['pT1']) & (df['pT1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['pT2']) & (df['pT2'] < self.pT_max)
        self.cuts += "\n" + r",  $%d <$ %s $< %d$ GeV" % (self.pT_min, self.p_str_latex, self.pT_max)  # The string brings in its own '$'.
        return mask_pT1, mask_pT2

    def get_mask_eta(self, df):
        mask_eta1 = (self.eta_min < abs(df['eta1'])) & (abs(df['eta1']) < self.eta_max)
        mask_eta2 = (self.eta_min < abs(df['eta2'])) & (abs(df['eta2']) < self.eta_max)   
        self.cuts += "\n" + r",  $%.2f < \left| \eta^{\mathrm{REC}} \right| < %.2f$" % (self.eta_min, self.eta_max)
        return mask_eta1, mask_eta2

    def get_mask_dR(self, df):
        mask_dR1 = (df['delta_R1'] < self.dR_cut)
        mask_dR2 = (df['delta_R2'] < self.dR_cut)
        self.cuts += "\n" + r",  $\Delta R < %.3f$" % (self.dR_cut)
        return mask_dR1, mask_dR2
    
    def get_mask_massZ(self, df):
        mask_massZ = (self.massZ_min < df['massZ']) & (df['massZ'] < self.massZ_max)
        self.cuts += r"$%.1f < m_{\mu\mu} < %.1f$ GeV" % (self.massZ_min, self.massZ_max)
        return mask_massZ    
    
    def apply_mask_get_data(self, kinem, lep_selection_type="", weave=False):
        """
        Return the kinematic lepton data which pass selections in this kinematic bin.
        E.g. Apply a boolean mask for event selection and retrieve all "delta_R1" values.

        Parameters
        ----------
        kinem : str
            A complete branch name in the DataFrame or root file. 
                E.g. "pT1", "genLep_pt2", "massZ"
        lep_selection_type : int
            The lepton's mask you want to apply.
            
            lep_selection_type = "1" -- get kinem values in which muon1 passes all selection criteria 
                                       (muon 2 may or may not pass selections).
            lep_selection_type = "2" -- get kinem values in which muon2 passes all selection criteria.
            lep_selection_type = "both" -- BOTH muons must pass selections to get data from event.
            lep_selection_type = "either" -- Either muon1, or muon2, or both must pass selections to get data from event.
            lep_selection_type = "independent" -- Get kinematic values for muon1 and muon2, with no restriction on which event they came from.
                Note: If kinem ends in "1" or "2", then the kinematic values of the other lepton are automatically grabbed.
        weave : bool
            Weave lep1 kinematic values and lep2 kinematic values together, so that slicing doesn't just give lep1. 
            Only relevant for "independent" selection.
        
        Returns
        -------
        kinem_vals : array 
            Kinematic values with chosen mask applied. 
            All values satisfy selection criteria for this kinematic bin.
        """
        df = self.binned_df
        mask1 = self.mask_kinembin_lep1
        mask2 = self.mask_kinembin_lep2
        
        if lep_selection_type == "1":
            # Only select events in which muon1 passes selections.
            mask = mask1
        elif lep_selection_type == "2":
            # Only select events in which muon2 passes selections.
            mask = mask2
        elif lep_selection_type == "both":
            # Only select events in which BOTH muon1 and muon2 pass selections.
            mask = mask1 & mask2
        elif lep_selection_type == "either":
            # Only select events in which BOTH muon1 and muon2 pass selections.
            mask = mask1 | mask2
        elif lep_selection_type == "independent":
            # Go through all muons in all events, without regard for other muon. 
            if kinem[-1] in ["1", "2"]:
                # Lep1 (or lep2) kinematic detected. Go find the other lepton's kinematic values.
                # FIXME: the variable massZ_vtxChi2 will be wrongly caught by this 'if' statement!
                kinem1 = kinem[:-1] + "1"
                kinem2 = kinem[:-1] + "2"
                kinem_vals1 = df[kinem1][mask1].values
                kinem_vals2 = df[kinem2][mask2].values
            else:
                # The kinematic doesn't depend on lep1 or lep2, like: massZ, GENmass2l, etc.
                kinem_vals1 = df[kinem][mask1].values
                kinem_vals2 = df[kinem][mask2].values
            
            if (weave):
                # Weave values together so that when slicing (like [:5]), you don't just grab kinem_vals1. 
                kinem_vals = np.array( weave_lists(kinem_vals1, kinem_vals2) )
            else: 
                kinem_vals = np.append(kinem_vals1, kinem_vals2)
        
        else: 
            raise RuntimeError("[ERROR] `lep_selection_type` was not specified properly. Stopping now.")
        
        return kinem_vals
    
    def make_2D_plot(self, 
                     x_kinem, y_kinem, 
                     x_bin_limits=[0, 1, 0.1], y_bin_limits=[0, 1, 0.1],
                     lep_selection_type="",
                     run_over_only_n_evts=-1, 
                     title="",
                     exclusive=True,
                     save_plot=False, save_as_png=False, verbose=False, outpath="",
                     ax=None):
        """
        Make a 2D plot. Two examples:
            (1) dphi vs. deta  
            (2) dphi vs. dtheta
        User can specify the binning along either axis. 
        This method plots only the muons which pass the selection 
        (as opposed to taking any event in which at least 1 muon pass kinematic bin criteria).
        
        Parameters
        ----------
        x_kinem : str
            The kinematical variable to be plotted along x-axis. 
            Only works for kinematics which end with '1' or '2'.
            - Example: x_kinem="delta_theta" (for which there are two branches: "delta_theta1", "delta_theta2")
        y_kinem : str
            The kinematical variable to be plotted along y-axis. 
            Only works for kinematics which end with '1' or '2'.
            - Example: y_kinem="delta_eta" (for which there are two branches: "delta_eta1", "delta_eta2")
        x_bin_limits : list or array-like of floats
            The bin limits on the horizontal axis. [bin_min_left_edge, bin_max_right_edge, bin_width]
            - Example: [-2.5, 2.5, 0.1]
        y_bin_limits : list or array-like of floats
            The bin limits on the vertical axis. [bin_min_left_edge, bin_max_right_edge, bin_width]
            - Example: [-2.5, 2.5, 0.1]
        lep_selection_type : str
            What kind of selection to perform on the leptons. Choices:
            #UPDATE
        run_over_only_n_evts : int
            Number of events to plot. Use '-1' to use all events in this kinembin.
        title : str
            Alternate title to put on plot. Overrides the default one made in this method.
        exclusive : bool
            Means "only put muons which passed all selections in this plot".
            FIXME: It must be set to True for now...
        save_plot : bool
            If True, save the plot as a pdf and possibly a png.
        save_as_png : bool
            If True, save the plot as a png.
        verbose : bool
            Get debug and code progress info.
        outpath : str
            Path to save plot.
            
        FIXME:
        - Maybe generalize this into a method to make any 2D plot.
        """           
        x_kinem1 = x_kinem + "1"
        x_kinem2 = x_kinem + "2"
        y_kinem1 = y_kinem + "1"
        y_kinem2 = y_kinem + "2"
        
#         x1_vals = self.apply_mask_get_data(x_kinem1, lep_selection_type="1")  # All x_kinem1 vals in which muon1 passed selection criteria. 
#         x2_vals = self.apply_mask_get_data(x_kinem2, lep_selection_type="2")  # All x_kinem2 vals in which muon2 passed selection criteria. 
#         y1_vals = self.apply_mask_get_data(y_kinem1, lep_selection_type="1")
#         y2_vals = self.apply_mask_get_data(y_kinem2, lep_selection_type="2")
        
        x_vals = self.apply_mask_get_data(x_kinem1, lep_selection_type="independent", weave=True)  # Should also be able to choose x_kinem2 with same effect.
        y_vals = self.apply_mask_get_data(y_kinem2, lep_selection_type="independent", weave=True)  # Should also be able to choose y_kinem1 with same effect.
        if run_over_only_n_evts != -1:
            x_vals = x_vals[:run_over_only_n_evts]
            y_vals = y_vals[:run_over_only_n_evts]
        
#         if run_over_only_n_evts == -1:
#             x_vals = np.append(x1_vals, x2_vals)
#             y_vals = np.append(y1_vals, y2_vals)
#         else: 
#             # Weave values together so they alternate; don't want to always run over muon1 values.
#             x_weave = weave_lists(x1_vals, x2_vals)
#             y_weave = weave_lists(y1_vals, y2_vals)
#             x_vals = np.array(x_weave)[:run_over_only_n_evts]
#             y_vals = np.array(y_weave)[:run_over_only_n_evts]

        # Old way. 
#         x_vals = self.apply_mask_get_data(x_kinem1, x_kinem2, run_over_only_n_evts=n, exclusive=exclusive)    
#         y_vals = self.apply_mask_get_data(y_kinem1, y_kinem2, run_over_only_n_evts=n, exclusive=exclusive)    
        
        # A special case to make comparison of (delta_phi vs. delta_theta) easy with (delta_phi vs. delta_eta).
        if (x_kinem1[:-1] == "delta_theta") and (y_kinem1[:-1] == "delta_phi"):
            x_vals *= -1

        #--- Make plots ---#
        if (ax is None):
            f, ax = plt.subplots(figsize=(12.8, 9.6))
        
        x_2D_bins, x_2D_bin_width = make_binning_array(x_bin_limits)
        y_2D_bins, y_2D_bin_width = make_binning_array(y_bin_limits) 
        
        # Plot 1: dphi vs. deta
        x_label_1 = label_LaTeX_dict[x_kinem1]["label"]
        x_label_2 = label_LaTeX_dict[x_kinem2]["label"]
        y_label_1 = label_LaTeX_dict[y_kinem1]["label"]
        y_label_2 = label_LaTeX_dict[y_kinem2]["label"]
        
        x_unit = label_LaTeX_dict[x_kinem2]["units"]
        y_unit = label_LaTeX_dict[y_kinem2]["units"]
        
        def prep_2D_label(label_1, label_2, unit, bin_width):
            label = "{},   {}".format(label_1, label_2)
            label += "\n" + "(bin width: {:.2E})".format(bin_width)
            if len(unit) > 0:
                label =label.rstrip(")")
                label += " {})".format(unit)  
            return label
        
        x_label = prep_2D_label(x_label_1, x_label_2, x_unit, x_2D_bin_width)
        y_label = prep_2D_label(y_label_1, y_label_2, y_unit, y_2D_bin_width)

        ax.set_xlabel(x_label)#, fontsize=label_size)
        ax.set_ylabel(y_label)#, fontsize=label_size)
        
        if len(title) > 0:
            title += "\n"
        cuts = "Selection:\n" + r"{}".format(self.cuts)
        ax.set_title(title + cuts)#, fontsize=label_size)
    
        # Stats: 
        stat_text_x = 0.82
        stat_text_y = 0.9
        
        stats_ls = get_stats_2Dhist(x_vals, y_vals)
        leg_label = make_stats_legend_for_2dhist(stats_ls)
        ax.text(stat_text_x, stat_text_y, leg_label, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        
        newcmp = change_cmap_bkg_to_white('rainbow')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x_vals, y_vals, bins=[x_2D_bins, y_2D_bins], cmap=newcmp)
        plt.colorbar(im, ax=ax)
        
        # Save plots, if you want. 
        file_name  = "2Dplot_dphi_vs_detaANDdtheta"
        file_name += "__%.2f_eta_%.2f" % (self.eta_min, self.eta_max)
        file_name += "__%d_%s_%d" % (self.pT_min, self.p_str, self.pT_max)
        
        save_plots_to_outpath(save_plot, outpath, file_name, save_as_png, verbose)


#     def apply_mask_get_data(self, kinem1, kinem2, run_over_only_n_evts=-1, exclusive=True):
#         """
#         Return the kinematic muon data which pass selections of this kinematic bin.
#         Example: Retrieve all "delta_R1" and "delta_R2" values after masks have been applied.

#         Parameters
#         ----------
#         kinem1 : str
#             A branch in the DataFrame or root file. Should correspond to muon1. 
#                 Examples: "pT1", "genLep_pt1"
#         kinem2 : str
#             A branch in the DataFrame or root file. Should correspond to muon2. 
#                 Examples: "pT2", "genLep_pt2"
#         run_over_only_n_evts : int
#             Number of events to run over. Use '-1' to specify all available events.
#         exclusive : bool
#             Means "only put muons which passed all selections in this plot".
#             FIXME: It must be set to True for now...
        
#         Returns
#         -------
#         kinem_vals : array 
#             kinem1 values (muon1) woven together with kinem2 values (muon2).
#             All values satisfy selection criteria for this kinematic bin.
#         """
#         df = self.binned_df
            
#         # Quick check.
#         if kinem1[:-1] != kinem2[:-1]:
#             raise ValueError("Problem! `kinem1` and `kinem2` are not the same kind of kinematical variable (e.g. `pT1` and `pT2`).\nStopping now.")

#         #--- Get data ---#
#         if (exclusive):   
#             # Keep only the muons which pass the kinem bin selection. 
#             vals_muon1 = df[kinem1][self.mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
#             vals_muon2 = df[kinem2][self.mask_kinembin_lep2].values  # Muon 2 passes kinem bin selection.

#             if run_over_only_n_evts == -1:
#                 # Running over all events. No need to slice.
#                 kinem_vals = np.append(vals_muon1, vals_muon2)
#             else:
#                 n = run_over_only_n_evts
#                 # Weave muon1 and muon2 values together in systematic way.
#                 # Otherwise muon1 values would mostly be selected when doing a slice, like [:n].
#                 weave_ls = weave_lists(vals_muon1, vals_muon2)
#                 kinem_vals = np.array(weave_ls)[:n]

#         else:
#             #--------# DEPRECATED FOR NOW.
#             # Keep both muons from each event in which AT LEAST ONE muon passed the kinem bin selection. 
#             raise RuntimeError("Stopping now. This section hasn't been fully developed.\nSet exclusive=True.")

#             kinem_vals = np.append(self.binned_df['delta_eta1'][:n].values, self.binned_df['delta_eta2'][:n].values)
#             y_vals = np.append(self.binned_df['delta_phi1'][:n].values, self.binned_df['delta_phi2'][:n].values)

#             x2_vals = np.append(self.binned_df['delta_theta1'][:n].values, self.binned_df['delta_theta2'][:n].values)
#             x2_vals = x2_vals * -1
#             y_vals = np.append(self.binned_df['delta_phi1'][:n].values, self.binned_df['delta_phi2'][:n].values)
#             #--------#

#         combined_key = "{} and {}".format(kinem1, kinem2)
#         self.kinem_vals_after_selection[combined_key] = kinem_vals

#         return kinem_vals


    def plot_kinem_genrec_comparison(self,
                              kinem_gen, kinem_rec, 
                              x_range_ls=[-1, 1],
                              bin_limits=[-0.5,0.5,0.5], 
                              ax=None, ax_ratio=None, log_scale=False
                              ):
            """
            FIXME: Need to implement under/overflow bins!

            Plots differences in kinematics. Includes a ratio plot at the bottom.
            
            Parameters
            ----------
            kinem_gen : str
                The generator-level kinematical variable from the column of DF.
                E.g. "genLep_pt1" or "genLep_eta2", etc.
            kinem_rec : str
                The reconsructed-level kinematical variable from the column of DF.
                E.g. "pT1" or "eta2", etc.
            x_range_ls : 2-element list
                The x_min and x_max to show along x_axis, for viewing purposes: [x_min, x_max]
            bin_limits : 3-element list
                [first_bin_left_edge, last_bin_right_edge, bin_width]
            ax : axes object
                An external axes object to pass in, on which the main plot will be plotted.
                If None, a default one will get created.
            ax_ratio : axes object
                An external axes object to pass in, on which the ratio plot will be plotted.
                If None, a default one will get created.
            log_scale : bool
                Set y-axis to log scale.
            """

            df = self.binned_df

            x_bin_arr, x_bin_width = make_binning_array(bin_limits)
            x_bin_centers_arr = shift_binning_array(x_bin_arr)

            lep = kinem_gen[-1]  # Get last character of kinematic. Example: pT1 --> 1. Should be a string!
            
            #--- Get data ---#
            if ("1" in kinem_gen) and ("1" in kinem_rec): 
                data_rec = df[kinem_rec][self.mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
                data_gen = df[kinem_gen][self.mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
            elif ("2" in kinem_gen) and ("2" in kinem_rec):
                data_rec = df[kinem_rec][self.mask_kinembin_lep2].values  # Muon 1 passes kinem bin selection.
                data_gen = df[kinem_gen][self.mask_kinembin_lep2].values  # Muon 1 passes kinem bin selection.
            else:
                err_msg = "\n    Either kinem_gen or kinem_rec does not end with a '1' or '2', or they are not the same as each other.\nStopping now."
                raise ValueError(err_msg)

            # Gen and Reco stats:
            stats_ls_gen = get_stats_1Dhist(data_gen)
            stats_ls_rec = get_stats_1Dhist(data_rec)

            #----------------#
            #--- Plot It. ---#
            #----------------#
            if (ax is None) or (ax_ratio is None):
                fig = plt.figure(figsize=(10,8))
                ax = fig.add_axes([0.17,0.33,0.825,0.54])  # [low_left_corner_x, low_left_corner_y, width, height]
                ax_ratio = fig.add_axes([0.17,0.12,0.825,0.20])

            leg_label_gen = make_stats_legend_for_1dhist(stats_ls_gen)
            leg_label_rec = make_stats_legend_for_1dhist(stats_ls_rec)

            leg_label_gen = leg_label_gen.replace("Entries", "GEN Entries")
            leg_label_rec = leg_label_rec.replace("Entries", "REC Entries")

            hist_bin_vals_gen, bin_edges_gen, _ = ax.hist(data_gen, bins=x_bin_arr, histtype='step', color='g', lw=2, label=leg_label_gen)
            hist_bin_vals_rec, bin_edges_rec, _ = ax.hist(data_rec, bins=x_bin_arr, histtype='step', color='b', label=leg_label_rec)

            hist_bin_vals_gen_modified = hist_bin_vals_gen.copy()
            hist_bin_vals_gen_modified[hist_bin_vals_gen == 0] = 0.0000000001
            ratio_vals = (hist_bin_vals_rec - hist_bin_vals_gen) / hist_bin_vals_gen_modified

            ax_ratio.errorbar(x_bin_centers_arr, ratio_vals, xerr=x_bin_width/2, color='black', ms=0, capsize=0, mew=0, ecolor='black', drawstyle='steps-mid', alpha=1)

            # Pretty up the plot. 
            ax_ratio.grid(which='major',axis='x')
            ax_ratio.grid(which='major',axis='y', ls='-')
            ax_ratio.grid(which='minor',axis='y')

            # Hide first tick label on y-axis, since it overlaps with ratio plot's tick label.
            a=ax.get_yticks().tolist()
            a[0]=''
            ax.set_yticklabels(a)

            # Hide main plot's x tick labels.
            plt.setp(ax.get_xticklabels(), visible=False)

            # Only show a few of the tick labels on ratio plot.
            n_tick_labels = 5
            ax_ratio.yaxis.set_major_locator(plt.MaxNLocator(n_tick_labels))
            ax_ratio.axhline(c='r', lw=2, ls='-')

            textsize_legend = 10
            textsize_axislabels = 12
            textsize_title = 12

            unit_gen = label_LaTeX_dict[kinem_gen]["units"]
            unit_rec = label_LaTeX_dict[kinem_rec]["units"]
            
            y_label = hist_y_label(x_bin_width, unit_gen)
            
            # Remember that ax shouldn't have an x-label; it's covered by the ax_ratio. 
            ax.set_ylabel(y_label, fontsize=textsize_axislabels)
            ax.set_title(r"Selection: {}".format(self.cuts), fontsize=textsize_title)
            
            plt.minorticks_on()

            x_min = x_range_ls[0]
            x_max = x_range_ls[1]
            ax.set_xlim([x_min, x_max])
            ax_ratio.set_xlim([x_min, x_max])
            ax_ratio.set_ylim([-0.12, 0.12])

            x_label_gen = label_LaTeX_dict[kinem_gen]["label"]
            x_label_rec = label_LaTeX_dict[kinem_rec]["label"]
            
            x_label = r"{}, {}".format(x_label_gen, x_label_rec)
            if len(unit_gen) > 0:
                x_label += " [{}]".format(unit_gen)
            ax_ratio.set_xlabel(x_label, fontsize=textsize_axislabels)
            ax_ratio.set_ylabel(r'$\frac{ (\mathrm{REC} - \mathrm{GEN}) }{ \mathrm{GEN} }$', fontsize=textsize_axislabels*1.5)

            y_max = max(max(hist_bin_vals_gen), max(hist_bin_vals_rec))
            ax.set_ylim([0, y_max*1.4])

            ax.legend(loc='upper right', framealpha=0.9, fontsize=textsize_legend)#, horizontalalignment='right')
            
            return ax, ax_ratio

    def plot_1D_kinematics(self, kinem="", lep_selection_type="independent", x_limits=[0,0], bin_limits=[0,0,0], run_over_only_n_evts=-1,
                           ax=None, x_label="", y_label="", title="", y_max=-1, log_scale=False, iter_gaus=(False, 0)):
        """
        Make a histogram of a kinematical variable in the DF.  
        FIXME: 
            - Currently only works with kinem variables that end with `1` or `2`. 
            [ ] Eventually plot other quantities like massZ, etc.
        
        Parameters
        ----------
        kinem : str
            The full name of the kinematical branch/column ("delta_eta1", "delta_R2", "massZ", etc.)
        lep_selection_type : int
            Indicates the kind of selection to apply on leptons:
                lep_selection_type = "1"    -- Plot kinem values in which only lepton1 passed selections.
                lep_selection_type = "2"    -- Plot kinem values in which only lepton2 passed selections.
                lep_selection_type = "both" -- Plot kinem values in which lepton1 AND lepton2 passed selections.
                lep_selection_type = "either" -- Plot kinem values in which lepton1 OR lepton2 passed selections.
        x_limits : 2-element list
            The x_min and x_max to show along x_axis, for viewing purposes: [x_min, x_max]
        bin_limits : 3-element list
            [first_bin_left_edge, last_bin_right_edge, bin_width]   
        run_over_only_n_evts : int
            Number of events to put into histogram. 
        ax : axes object
            An external axes object to pass in, on which the main plot will be plotted.
            If None, a default one will get created.
        x_label : str
            Override the default x-label that would be produced.
        y_label : str
            Override the default y-label that would be produced.
        title : str
            Override the default title that would be produced.
        y_max : float
            The max y-value on vertical axis for viewing purposes.
            If set to -1, then automatic detection is used. 
        log_scale : bool
            Use log-scale on vertical axis.
        iter_gaus : 2-tuple
            If True, perform an iterative gaussian fit on the core N times.
            Syntax: (switch, N)
        """
        df = self.binned_df
        

        
#         x_kinem1 = x_kinem + "1"
#         x_kinem2 = x_kinem + "2"
#         y_kinem1 = y_kinem + "1"
#         y_kinem2 = y_kinem + "2"
        
#         x1_vals = self.apply_mask_get_data(x_kinem1, lep_selection_type="1")  # All x_kinem1 vals in which muon1 passed selection criteria. 
#         x2_vals = self.apply_mask_get_data(x_kinem2, lep_selection_type="2")  # All x_kinem2 vals in which muon2 passed selection criteria. 
#         y1_vals = self.apply_mask_get_data(y_kinem1, lep_selection_type="1")
#         y2_vals = self.apply_mask_get_data(y_kinem2, lep_selection_type="2")
        
#         if run_over_only_n_evts == -1:
#             x_vals = np.append(x1_vals, x2_vals)
#             y_vals = np.append(y1_vals, y2_vals)
#         else: 
#             # Weave values together so they alternate; don't want to always run over muon1 values.
#             x_weave = weave_lists(x1_vals, x2_vals)
#             y_weave = weave_lists(y1_vals, y2_vals)
#             x_vals = np.array(x_weave)[:run_over_only_n_evts]
#             y_vals = np.array(y_weave)[:run_over_only_n_evts]

        
        
        
#         if lep_selection_type in ["either", "both"]:
#             # Must look at 
        
#         if kinem[:1] not in ["1","2"]:
#             # THEN THIS IS A SPECIAL KINEMATICAL VARIABLE
            
#         if lep_selection_type == 1: 
#             data = df[kinem][self.mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
#         elif lep_selection_type == 2:
#             data = df[kinem][self.mask_kinembin_lep2].values  # Muon 2 passes kinem bin selection.
#         elif lep_selection_type == -1: 
#             data = df[kinem][self.mask_kinembin_lep1].values
#         else:
#             err_msg = "\n    You  must specify lep = 1 or 2.\nStopping now."
#             raise ValueError(err_msg)
            
#         self, kinem, lep_selection_type="", weave=False
            
            
            
            
            
            
            
            
        #--- Get data ---#
        data = self.apply_mask_get_data(kinem=kinem, lep_selection_type=lep_selection_type, weave=True)  # kinem must be a full name
            
        if run_over_only_n_evts != -1:
            data = data[:run_over_only_n_evts]
            
        if ax is None:
            # Axes doesn't exist yet. Make it.
            fig, ax = plt.subplots(figsize=(12.8,9.6))
            
        if bin_limits == [0,0,0]:
            # No bin limits specified, so use default binning for this kinematical variable.
            bin_limits = label_LaTeX_dict[kinem]["default_bin_limits"]
             
        if x_limits == [0,0]:
            # No x-limits specified, so use default x-limits for this kinematical variable.
            x_limits = label_LaTeX_dict[kinem]["default_x_limits"]
            
        unit = label_LaTeX_dict[kinem]["units"]
            
        x_bins, binw = make_binning_array(bin_limits)

        if len(x_label) == 0:
                # Both muons are being chosen. Change the labels.
            key = "independent_label" if lep_selection_type == "independent" else "label"
            x_label = label_LaTeX_dict[kinem][key]
            if len(unit) > 0:
                x_label += " [{}]".format(unit)
            
        if len(y_label) == 0:
            # User didn't specify label, so make it.
            y_label = hist_y_label(binw, unit)
            
        if len(title) == 0:
            title = "Selections:\n" + self.cuts
        
        ax, bin_vals, bin_edges, stats = make_1D_dist(ax, data, x_limits, x_bins, x_label, y_label, title, y_max=-1, log_scale=False)
        
        if (iter_gaus[0]):
            # Do iterative fitting procedure.
            # Produce a dictionary of stats for these fits.
            stats_dict, ax = iterative_fit_gaus(iter_gaus[1], bin_edges, bin_vals, first_mean=stats[1], first_stdev=stats[3], ax=ax)
            # Use plotted kinem as the key for this dict of stats. 
            self.stats_dict[kinem] = stats_dict         
            
    def count_in_pT_eta_region_exclusive(self):
        """
        Return the number of muons in the DF subset which pass the eta and pT cuts. 
        'Exclusive' because only those muons which pass such cuts are counted 
        (as opposed to counting both muons from event just because one pass the cuts).
        """
        pass


In [ ]:
# Variables which aren't specifically related to muon1 or muon2. 

# massZ, massZErr, 'massZ_vtx', 'massZ_vtx_FSR', 'massErrZ_vtx',
#        'massErrZ_vtx_FSR', 'massZ_vtxChi2', 'massZ_vtx_BS', 'massZ_vtx_BS_FSR',
#        'massErrZ_vtx_BS', 'massErrZ_vtx_BS_FSR', 'massZ_vtxChi2_BS',
#         'weight',
#        'GENmass2l','nFSRPhotons'

In [ ]:
label_LaTeX_dict = {
    "pT1"  : {"label":r"$p_{T1}^{\mathrm{REC}}$", 
              "independent_label":r"$p_{T}^{\mathrm{REC}}$", 
              "units":"GeV"},
    
    "pT2"  : {"label":r"$p_{T2}^{\mathrm{REC}}$", 
              "independent_label":r"$p_{T}^{\mathrm{REC}}$", 
              "units":"GeV"},
    
    'eta1' : {"label":r"$\eta_{1}^{\mathrm{REC}}$", 
              "independent_label":r"$\eta^{\mathrm{REC}}$", 
              "units":""},
    
    'eta2' : {"label":r"$\eta_{2}^{\mathrm{REC}}$", 
              "independent_label":r"$\eta^{\mathrm{REC}}$", 
              "units":""},
    
    'theta1' : {"label":r"$\theta_{1}^{\mathrm{REC}}$", 
              "independent_label":r"$\theta^{\mathrm{REC}}$", 
                "units":""},
    
    'theta2' : {"label":r"$\theta_{2}^{\mathrm{REC}}$", 
              "independent_label":r"$\theta^{\mathrm{REC}}$", 
                "units":""},
    
    'phi1' : {"label":r"$\phi_{1}^{\mathrm{REC}}$", "units":""},
    'phi2' : {"label":r"$\phi_{2}^{\mathrm{REC}}$", "units":""},
    
    'genLep_pt1' : {"label":r"$p_{T1}^{\mathrm{GEN}}$", "units":"GeV"},
    'genLep_pt2' : {"label":r"$p_{T2}^{\mathrm{GEN}}$", "units":"GeV"},
    
    'genLep_eta1' : {"label":r"$\eta_{1}^{\mathrm{GEN}}$", "units":""},
    'genLep_eta2' : {"label":r"$\eta_{2}^{\mathrm{GEN}}$", "units":""},
    
    'genLep_phi1' : {"label":r"$\phi_{1}^{\mathrm{GEN}}$", "units":""},
    'genLep_phi2' : {"label":r"$\phi_{2}^{\mathrm{GEN}}$", "units":""},
    
    'genLep_theta1' : {"label":r"$\theta_{1}^{\mathrm{GEN}}$", "units":""},
    'genLep_theta2' : {"label":r"$\theta_{2}^{\mathrm{GEN}}$", "units":""},
    
    "delta_pT1" : {"label":r"$\Delta p_{T1} \equiv p_{T1}^{\mathrm{REC}} - p_{T1}^{\mathrm{GEN}} $", "units":"GeV"},
    "delta_pT2" : {"label":r"$\Delta p_{T2} \equiv p_{T2}^{\mathrm{REC}} - p_{T2}^{\mathrm{GEN}} $", "units":"GeV"},
    
    "delta_pToverpT1"    : {"label":r"$ \Delta p_{T1} \ / p_{T1}^{\mathrm{GEN}}$", 
                            "independent_label":r"$ \Delta p_{T} \ / p_{T}^{\mathrm{GEN}}$", 
                            "units":""},
    "delta_pToverpT2"    : {"label":r"$ \Delta p_{T2} \ / p_{T2}^{\mathrm{GEN}}$", 
                            "independent_label":r"$ \Delta p_{T} \ / p_{T}^{\mathrm{GEN}}$", 
                            "units":""},
    "delta_pToverRecpT1" : {"label":r"$ \Delta p_{T1} \ / p_{T1}^{\mathrm{REC}}$", 
                            "independent_label":r"$ \Delta p_{T} \ / p_{T}^{\mathrm{GEN}}$", 
                            "units":""},
    "delta_pToverRecpT2" : {"label":r"$ \Delta p_{T2} \ / p_{T2}^{\mathrm{REC}}$", 
                            "independent_label":r"$ \Delta p_{T} \ / p_{T}^{\mathrm{GEN}}$", 
                            "units":""},
    
#     "delta_pToverpT2_squared" : {"label":r"$( p_{T}^{\mathrm{REC}} - p_{T}^{\mathrm{GEN}} ) / (p_{T}^{\mathrm{GEN}})^2 $ GeV$^{-1}$", "units":r"GeV$^{-1}$"},
    
    "delta_R1" : {"label":r"$\Delta R_{1} = \sqrt{  (\Delta \eta_{1})^2 + (\Delta \phi_{1})^2   }$", "units":""},
    "delta_R2" : {"label":r"$\Delta R_{2} = \sqrt{  (\Delta \eta_{2})^2 + (\Delta \phi_{2})^2   }$", "units":""},
    
    "delta_eta1" : {"label":r"$\Delta \eta_{1} = \eta_{1}^{\mathrm{REC}} - \eta_{1}^{\mathrm{GEN}}$", "units":""},
    "delta_eta2" : {"label":r"$\Delta \eta_{2} = \eta_{2}^{\mathrm{REC}} - \eta_{2}^{\mathrm{GEN}}$", "units":""},
    
    "delta_theta1" : {"label":r"$\Delta \theta_{1} = \theta_{1}^{\mathrm{REC}} - \theta_{1}^{\mathrm{GEN}}$", "units":""},
    "delta_theta2" : {"label":r"$\Delta \theta_{2} = \theta_{2}^{\mathrm{REC}} - \theta_{2}^{\mathrm{GEN}}$", "units":""},
    
    "delta_phi1" : {"label":r"$\Delta \phi_{1} = \phi_{1}^{\mathrm{REC}} - \phi_{1}^{\mathrm{GEN}}$", "units":""},
    "delta_phi2" : {"label":r"$\Delta \phi_{2} = \phi_{2}^{\mathrm{REC}} - \phi_{2}^{\mathrm{GEN}}$", "units":""},
    
    "d0BS1" : {"label":r"$d_{0, \mathrm{lep1}}^{ \mathrm{BS} }$", 
              "independent_label":r"$d_{0}^{ \mathrm{BS} }$", 
               "units":"cm"},
    "d0BS2" : {"label":r"$d_{0, \mathrm{lep2}}^{ \mathrm{BS} }$", 
              "independent_label":r"$d_{0}^{ \mathrm{BS} }$", 
               "units":"cm"},
    "d0PV1" : {"label":r"$d_{0, \mathrm{lep1}}^{ \mathrm{PV} }$", 
              "independent_label":r"$d_{0}^{ \mathrm{PV} }$", 
               "units":"cm"},
    "d0PV2" : {"label":r"$d_{0, \mathrm{lep2}}^{ \mathrm{PV} }$", 
              "independent_label":r"$d_{0}^{ \mathrm{PV} }$", 
               "units":"cm"},
    
    "d0BSq1" : {"label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu_{1}^{REC})$", 
              "independent_label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu^{REC})$", 
                 "units":"cm"},
    "d0BSq2" : {"label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu_{2}^{REC})$", 
              "independent_label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu^{REC})$", 
                 "units":"cm"},
    "d0PVq1" : {"label":r"$d_{0}^{ \mathrm{PV} } * \mathrm{charge}(\mu_{1}^{REC})$", 
              "independent_label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu^{REC})$", 
                 "units":"cm"},
    "d0PVq2" : {"label":r"$d_{0}^{ \mathrm{PV} } * \mathrm{charge}(\mu_{2}^{REC})$", 
              "independent_label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu^{REC})$", 
                 "units":"cm"},
    
    "massZ"    : {"label":r"$m_{2\mu}$",        "units":"GeV", "default_bin_limits":[70,110,0.5], "default_x_limits":[70, 110]},
    "massZErr" : {"label":r"$\delta m_{2\mu}$", "units":"GeV", "default_bin_limits":[0,2,0.05],   "default_x_limits":[-0.1, 2.1]},

    
# Unused branches: 
# ', 'massZErr', 'massZ_vtx', 'massZ_vtx_FSR', 'massErrZ_vtx',
#        'massErrZ_vtx_FSR', 'massZ_vtxChi2', 'massZ_vtx_BS',
#        'm1', 'm2', 'Id1', 'Id2', 'Tight1', 'Tight2', 'pterr1', 'pterr2', 'weight','GENmass2l', 
#         'nFSRPhotons',  'genLep_p1', 'genLep_p2', 'p1', 'p2', 
#         'delta_Rtheta1', 'delta_Rtheta2', 'delta_pToverGenpT2'
}

## Make the $d_0 * q$ distributions!

In [ ]:
# f, ax = plt.subplots(figsize=(12.8,9.6))

n_evts_scan = 100000
n_evts_keep = 10000

kbin = KinemBinnedEtaPt(df_MC_2016, 
                        n_evts=n_evts_scan, 
                        eta_cut_ls=[0.0, 0.3], 
                        pT_cut_ls=[5, 100], 
                        use_ptotal_instead=False, 
                        dR_cut=0.02, verbose=True)

In [ ]:
# kbin.plot_1D_kinematics(kinem="d0BSxq1", lep_selection_type='independent', x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], run_over_only_n_evts=n_evts_keep, ax=None, y_max=-1, log_scale=False, iter_gaus=(False, 3))

fig = plt.figure(figsize=(25.6,19.2))
ax = plt.subplot(221)
kbin.plot_1D_kinematics(kinem="delta_pToverpT1", lep_selection_type='independent', x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], run_over_only_n_evts=-1, ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 5))
ax = plt.subplot(222)
kbin.plot_1D_kinematics(kinem="delta_pToverpT2", lep_selection_type='independent', x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], run_over_only_n_evts=-1, ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 5))
ax = plt.subplot(223)
kbin.plot_1D_kinematics(kinem="delta_pToverRecpT1", lep_selection_type='independent', x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], run_over_only_n_evts=-1, ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 5))
ax = plt.subplot(224)
kbin.plot_1D_kinematics(kinem="delta_pToverRecpT2", lep_selection_type='independent', x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], run_over_only_n_evts=-1, ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 5))


In [ ]:
%%time
# %%capture

n_evts_scan = 10000
n_evts_keep = 1000

eta_bin_edges = [0.00, 0.20] # barrel
# eta_bin_edges = [0.80, 1.10]  # overlap
# eta_bin_edges = [2.10, 2.40]  # endcap

# eta_bin_edges = [0.00, 0.10, 0.20, 0.30]    # barrel
# eta_bin_edges = [0.70, 0.80, 0.90, 1.00, 1.10, 1.20]    # overlap
# eta_bin_edges = [2.00, 2.10, 2.20, 2.30, 2.40]    # endcap

#--- Could be either pT or p. User specifies when initializing kbin. ---#
# p_bin_edges = [5, 20, 30, 40, 50, 60, 100]
# p_bin_edges = [5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100]
# p_bin_edges = [5, 7, 10, 15, 20]
p_bin_edges = [5, 20, 30]
# p_bin_edges = [20, 40, 60, 80, 100]
use_ptotal_instead = False  # p_total or pT

outpath = "/Users/Jake/Desktop/20200423/2Dplots_dPhivsdEtaANDdTheta/"
pdf_name_base = "TEST1"

save_plot = False  # Save individual plots.
save_as_png = False
save_pdf = True  
verbose = True

#------------------#
#--- Automatons ---#
#------------------#
if (save_plot) and (save_pdf or save_as_png):
    err_msg = "PROGRAM STOPPED: both 'save_plot' and 'save_pdf' are True. Make one of them False."
    raise RuntimeError(err_msg)

makeDirs(outpath)
all_kbin_ls = []
for k in range(len(eta_bin_edges)-1):
    this_eta = eta_bin_edges[k]
    next_eta = eta_bin_edges[k+1]
    
    kbin_ls = []
    for m in range(len(p_bin_edges)-1):
        this_p = p_bin_edges[m]
        next_p = p_bin_edges[m+1]

        kbin = KinemBinnedEtaPt(df_MC_2016, 
                                n_evts=n_evts_scan, 
                                eta_cut_ls=[this_eta, next_eta], 
                                pT_cut_ls=[this_p, next_p], 
                                use_ptotal_instead=False, 
                                dR_cut=0.02, verbose=verbose)
        kbin_ls.append(kbin)
        
    # Finished looping over p bins.
    evts_max = n_evts_keep

    if save_pdf and not save_plot:
        # Save plots into one PDF:
        pT_min = min( [kb.pT_min for kb in kbin_ls] )
        pT_max = max( [kb.pT_max for kb in kbin_ls] )
        n_plots = len(kbin_ls)

        title_str_pT_min = f"{pT_min}" if pT_min < 10 else f"{pT_min}"  # For plot-ordering purposes.
        pdf_name = pdf_name_base + f"__{this_eta}_eta_{next_eta}"# + f"__{title_str_pT_min}_pT_{pT_max}"
        pdf_name += f"__{pT_min}_pT_{pT_max}"# + f"__{title_str_pT_min}_pT_{pT_max}"
        pdf_name += f"__{n_plots}plots"
        pdf_name = make_str_title_friendly(pdf_name) + ".pdf"

        outfile = os.path.join(outpath, pdf_name)

        with PdfPages(outfile) as pdf:
            for kbinned in kbin_ls:
                kbinned.make2Dplot_dPhi_vs_dEtaANDdTheta(run_over_only_n_evts=evts_max,
                                                           x1_bounds=[-0.003, 0.003, 0.00003], 
                                                           x2_bounds=[-0.003, 0.003, 0.00003], 
                                                           y_bounds=[-0.003, 0.003, 0.00003], 
                                                           exclusive=True,
                                                           save_plot=save_plot, 
                                                           save_as_png=save_as_png,
                                                           verbose=verbose,
                                                           outpath=outpath
                                                           )

                kbinned.make_1D_dist_dPhi()
                kbinned.make_1D_dist_dEta()
                kbinned.make_1D_dist_dR()
                kbinned.make_1D_dist_dTheta()

                pdf.savefig()  # saves the current figure into a pdf page
                plt.close()

        print("[INFO] PDF created at", outfile, "\n")

        all_kbin_ls.append(kbin_ls)

        plt.close('all')
        
    elif save_plot and not save_pdf:
        # Save individual plots:
        for kbinned in kbin_ls:
            kbinned.make2Dplot_dPhi_vs_dEtaANDdTheta(run_over_only_n_evts=evts_max,
                                                       x1_bounds=[-0.003, 0.003, 0.00003], 
                                                       x2_bounds=[-0.003, 0.003, 0.00003], 
                                                       y_bounds=[-0.003, 0.003, 0.00003], 
                                                       exclusive=True,
                                                       save_plot=save_plot, 
                                                       save_as_png=save_as_png,
                                                       verbose=verbose,
                                                       outpath=outpath
                                                       )
            plt.close('all')

### Loop over $p_{T}$ vs. $\eta$ plots

In [ ]:
n_evts = 100
eta_bin_edges = [0.0, 0.9, 1.4, 2.4] 
# eta_bin_edges = [0.9, 1.4, 2.4] 
# eta_bin_edges = [0.9, 1.4, 2.4] 
pT_bin_edges = [5, 20, 30, 40, 50, 60, 100]
# pT_bin_edges = [30, 40, 50]
# pT_bin_edges = [40, 50]
eta_2D_limits = [-2.5, 2.5, 0.1] 
pT_2D_limits = [0, 100, 1] 
save_plot = True 
outpath = "/Users/Jake/Desktop/20200424/2Dplots_pT_vs_eta_1E2evts_Test/"

makeDirs(outpath)
                            
for k in range(len(eta_bin_edges)-1):
    this_eta = eta_bin_edges[k]
    next_eta = eta_bin_edges[k+1]

    for m in range(len(pT_bin_edges)-1):
        this_pT = pT_bin_edges[m]
        next_pT = pT_bin_edges[m+1]

        kbin = KinemBinnedDataFrame(df_MC_2016, n_evts=n_evts, eta_cut_ls=[this_eta, next_eta], pT_cut_ls=[this_pT, next_pT], dR_cut=0.002)
        # BELOW IS DEPRECATED
        kbin.make_2D_plot(eta_2D_limits=eta_2D_limits, pT_2D_limits=pT_2D_limits, save_plot=True, outpath=outpath)
        
        make_2D_plot(self, 
                     x_kinem, y_kinem, 
                     x_bin_limits=[0, 1, 0.1], y_bin_limits=[0, 1, 0.1],
                     lep_selection_type="",
                     run_over_only_n_evts=-1, 
                     title="",
                     exclusive=True,
                     save_plot=False, save_as_png=False, verbose=False, outpath="",
                     ax=None):